<a href="https://colab.research.google.com/github/ryanleeallred/DS-Unit-4-Sprint-1-NLP/blob/main/module3-document-classification/LS_DS_413_Document_Classification_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lectures.

Today's guided module project and assignment will be different. You already know how to do classification. You already know how to extract features from documents. So? That means you're ready to combine and practice those skills in a Kaggle competition. We will open with a five-minute sprint explaining the competition and then give you 25 minutes to work. After those twenty-five minutes are up, I will give a 5-minute demo of an NLP technique to help you with document classification (*and **maybe** the competition*).

Today is all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [1]:
import pandas as pd
import re

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
import xgboost as xgb
from xgboost import XGBClassifier

from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
import spacy
nlp = spacy.load("en_core_web_md")

In [43]:
# You may need to change the path
train = pd.read_csv('train.csv',usecols=['description','category'])
test = pd.read_csv('test.csv',usecols=['description'])

submission_test=pd.read_csv('test.csv')

In [3]:
train.head()

,description,category
0,A marriage of 13 and 18 year old bourbons. A m...,2
1,There have been some legendary Bowmores from t...,1
2,This bottling celebrates master distiller Park...,2
3,What impresses me most is how this whisky evol...,1
4,"A caramel-laden fruit bouquet, followed by une...",2


### Define Pipeline Components

In [17]:
def clean_text(text):
  # COMPLETE THE CODE IN THIS CELL
  # remove new line characters
  text = text.replace('\\n', ' ')

  # remove numbers for the text
  non_alpha = '[^a-zA-Z]'
  multi_white_spaces = "[ ]{2,}"
  
  text = re.sub(non_alpha, ' ', text)
  text = re.sub(multi_white_spaces, " ", text)

  # case normalize and strip extra white spaces on the far left and right hand side
  return text.lower().lstrip().rstrip()

train['description'] = train['description'].apply(clean_text)
test['description'] = test['description'].apply(clean_text)

In [20]:
# prep data, instantiate a model, create pipeline object, and run a gridsearch 

###BEGIN SOLUTION

target = 'category'
# COMPLETE THE CODE IN THIS CELL
y, X = train[target], train.drop(columns=target)

X = [list(v)[0] for v in X.values]

# Create Pipeline Components

# create vectorizer
tfidf = TfidfVectorizer(stop_words="english", tokenizer=None) # data transformer 

# create classifier
rfc = RandomForestClassifier(random_state=42) # estimator 

# Instantiate a pipeline object
pipe = Pipeline([("vect", tfidf), # data transformer
                 ("clf", rfc)])   # classifier 

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [23]:
parameters = {
    'vect__max_df': (0.75, 1.0),
    'clf__max_depth':(5,10,15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=4, verbose=1)
grid_search.fit(X, y)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(stop_words='english')),
                                       ('clf',
                                        RandomForestClassifier(random_state=42))]),
             n_jobs=4,
             param_grid={'clf__max_depth': (5, 10, 15, 20),
                         'vect__max_df': (0.75, 1.0)},
             verbose=1)

In [26]:
grid_search.best_score_

0.7757085352830033

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition, the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [24]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [44]:
submission = pd.DataFrame({'id': submission_test['id'], 'category':pred})
submission['category'] = submission['category'].astype('int64')

In [45]:
# Make Sure the Category is an Integer
submission.head()

,id,category
0,955,2
1,3532,2
2,1390,1
3,1024,1
4,1902,1


In [49]:
subNumber = 0

In [52]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'.submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

You're trying to achieve a minimum of 80% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (LSI) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores i.e., `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [28]:
svd = TruncatedSVD(n_components=2, # number of topics to generate (also the size of the new feature space)
                   algorithm='randomized',
                   n_iter=10)

vect = TfidfVectorizer(stop_words="english", tokenizer=None)

lsi = Pipeline([("vect", vect), # creating our term-doc matrix
                ("svd", svd)]) # apply svd to our term-doc matrix 

clf = XGBClassifier()

pipe = Pipeline([('lsi', lsi), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [29]:
parameters = {
    'lsi__svd__n_components': [10,100,250],
    'lsi__vect__max_df': (0.75, 1.0),
    'clf__max_depth':(5,10,15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=4, verbose=1)
grid_search.fit(X, y)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


C:\Users\nigel\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[01:06:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('lsi',
                                        Pipeline(steps=[('vect',
                                                         TfidfVectorizer(stop_words='english')),
                                                        ('svd',
                                                         TruncatedSVD(n_iter=10))])),
                                       ('clf',
                                        XGBClassifier(base_score=None,
                                                      booster=None,
                                                      colsample_bylevel=None,
                                                      colsample_bynode=None,
                                                      colsample_bytree=None,
                                                      gamma=None, gpu_id=None,
                                                      importance_type='gain',
                                               

### Make a Submission File

In [30]:
grid_search.best_score_

0.9176299261405644

In [31]:
print(grid_search.best_params_)

{'clf__max_depth': 5, 'lsi__svd__n_components': 100, 'lsi__vect__max_df': 0.75}


In [ ]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [46]:
submission = pd.DataFrame({'id': submission_test['id'], 'category':pred})
submission['category'] = submission['category'].astype('int64')

In [47]:
# Make Sure the Category is an Integer
submission.head()

,id,category
0,955,2
1,3532,2
2,1390,1
3,1024,1
4,1902,1


In [54]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'.submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

## Follow Along

In [33]:
# Apply to your Dataset

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier

from scipy.stats import randint

param_dist = {
    
    'max_depth' : randint(3,10),
    'min_samples_leaf': randint(2,15)
}

In [34]:
# Continue Word Embedding Work Here
nlp = spacy.load("en_core_web_md")

def get_word_vectors(text):
    """
    This serves as both our tokenizer and vectorizer. 
    Returns a list of word vectors, i.e. our doc-term matrix
    """
    return [nlp(doc).vector for doc in text]

X_train_emb = get_word_vectors(train['description'])
X_test_emb = get_word_vectors(test['description'])

In [41]:
rfc = RandomForestClassifier(oob_score=True)

rfc.fit(X_train_emb, y)

RandomForestClassifier(oob_score=True)

In [42]:
print('Training Accuracy: ', rfc.score(X_train_emb, y))

Training Accuracy:  1.0


In [37]:
rfc.oob_score_

0.7269914926527455

### Make a Submission File

In [ ]:
# Predictions on test sample
pred = ...predict(test['description'])

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (LSI) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores i.e., `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Submit to Kaggle 

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 80% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis." Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis?" 
    - Is Document Classification different than "Sentiment Analysis?" Provide evidence for your response
    - How do you create labeled sentiment data? Are those labels sentiment?
    - What are common applications of sentiment analysis?
2. Research why word embeddings worked better for the lecture notebook than for the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest.
    - Neural Networks are becoming more popular for document classification. Why is that the case?